### Import package

In [2]:
#LiveLattice
from livelattice.processing import *
from livelattice.pipeline import *
import sys
from PyQt5.QtWidgets import QFileDialog, QApplication
from sld import SlideBook

%load_ext autoreload
%autoreload 2
#CAMDU
import tifffile
import napari

/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/pycudadecon/_libwrap.py:45: UserWarning: Unable to find function: camcor_interface_init in libcudaDecon
  def camcor_interface_init(  # type: ignore [empty-body]
/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/pycudadecon/_libwrap.py:52: UserWarning: Unable to find function: camcor_interface in libcudaDecon
  def camcor_interface(  # type: ignore [empty-body]


### Select the folders to process using automatic PSF detection

In [7]:
#number_of_dates = 1

# Linux
#root_path = '/run/user/1000/gvfs/smb-share:server=taurus0.jslab.ucsd.edu,share=data/MOSAIC_Data/Data/'
#output_path = '/run/user/1000/gvfs/smb-share:server=taurus0.jslab.ucsd.edu,share=processing/MOSAIC_Data/Processed_Data/'

#app = QApplication(sys.argv)
#all_data_path = []
#for index in range(number_of_dates):
#    data_path = QFileDialog.getExistingDirectory(None, "Select data directory #"+str(index+1)+" ...", root_path).split('/')[-1]
#    all_data_path.append(data_path)

#tif_path_list, save_path_list = collect_data_locations(root_path, all_data_path, output_path)

#specified_channels = None # None for auto detection, use a list of wave numbers (string) if to manually specify, e.g. ['488', '642']

#selected_psf_path = None # None for auto detection, use a list of PSF paths if to manually specify

#Windows
#image_path='E:\LauraCooper\MWA\Raw-data-example-5D.sld - Raw_data_slide3_capture12-1.tif'
#image_path='E:\LauraCooper\MWA\Raw-data-example-5D.sld - Raw_data_slide3_capture12-1 every4thslice.tif'
#psf_path='E:\LauraCooper\MWA\PSFs-as-tiffs\\2024-09-13_PSF_488.tif'

#Linux
image_path = '/run/user/1675160/gvfs/smb-share:server=ads.warwick.ac.uk,share=shared/Shared221/CAMDU/Grants/MonashWarwickAlliance/Warwick3i/sldy-examples-from-2022/PSFs.sldy'
psf_path = '/run/user/1675160/gvfs/smb-share:server=ads.warwick.ac.uk,share=shared/Shared221/CAMDU/Grants/MonashWarwickAlliance/Warwick3i/sldy-examples-from-2022/PSFs/PSF_CH0.tif'

if image_path.lower().endswith(".tif"):
    image=tifffile.imread(image_path)
    print(image.shape)
elif image_path.lower().endswith(".sldy"):
    sld = SlideBook(image_path)
    print(sld.number_acquisitions)
    print(sld.images[0].num_channels)
    image = np.squeeze(sld.images[7].data["ch_0"])
    print(image.shape)
else:
    print("Unsupported file type")

psf_stack=tifffile.imread(psf_path)
viewer = napari.Viewer()
viewer.add_image(image, name='image', colormap='gray')
napari.run()

12
1
(101, 64, 64)


### Set parameters and process the movies

In [8]:
# select specific folders and/or frames within each folder to process
# use None if to process all
# index starts with 0
folders_to_process = None
frames_to_process = None

# options
save_mip = True # whether to create the MIP folder with individual and combined MIP images
overwrite_file = True # whether to overwrite exisiting files, use False if continue previous run
use_dask = False

# select operations to perform
num_decon_it = 15
decon_zsection_size = 400 # to avoid memory overflow, may lower if deconvolution is slow
decon_deskew_rotate = True
no_decon = False
decon_only = False

# run the processing
#process_data(tif_path_list, save_path_list,
#             specified_channels, selected_psf_path,
#             folders_to_process, frames_to_process,
#             save_mip, overwrite_file, use_dask,
#             num_decon_it, decon_zsection_size, 
#             decon_deskew_rotate, no_decon, decon_only)
processing = image

In [9]:
t1 = time.time()
num_zsections = 0
if image.shape[0] % decon_zsection_size == 0:
    num_zsections = int(image.shape[0] / decon_zsection_size)
else:
    num_zsections = int(image.shape[0] / decon_zsection_size) + 1

pad_size = int(0.1*decon_zsection_size)
start_z = pad_size
full_deconv = None

for ind_z in range(num_zsections):
      
    section = processing[start_z-pad_size:start_z+decon_zsection_size+pad_size]
    if section.shape[0] < pad_size:
        break
                
    print('\nDeconvolution starts for z-section', ind_z+1)
    with TemporaryOTF(psf=psf_path, dzpsf=dzpsf, dxpsf=dxpsf, fixorigin=0, otf_bgrd=0, max_otf_size=None, skewed_decon=True) as otf:
        with RLContext(section.shape, otf.path, dzdata, dxdata, dzpsf, dxpsf, skewed_decon=True) as ctx:
                
            section_deconv = rl_decon(section, background=0, n_iters=num_decon_it, skewed_decon=True)

            # to avoid boundary effect due to partitioning in all x,y,z, we divide only z in blocks for large dataset
            if start_z == pad_size:
                full_deconv = section_deconv[:pad_size+decon_zsection_size]
            else:
                full_deconv = np.concatenate([full_deconv, section_deconv[pad_size:pad_size+decon_zsection_size]])

            start_z += decon_zsection_size
                    
    #processing = full_deconv.astype('uint16')


Deconvolution starts for z-section 1
callednx=64, ny=64, nz=101
Center of mass is (15.079, 48.153, 40.368)
Background is 107.236
output nz=100
X_k allocated.                 1MB   10776MB free Pinning raw.data's Host RAM.  Copy raw.data to X_k HostToDevice.  Done.  
CC allocated.                  1MB   10774MB free
rawGPUbuf allocated.           1MB   10772MB free
X_kminus1 allocated.           1MB   10770MB free
Y_k allocated.                 1MB   10768MB free
G_kminus1 allocated.           1MB   10766MB free
G_kminus2 allocated.           1MB   10764MB free
fftGPUbuf allocated.           1MB   10762MB free
Iteration  0. Cpy X_k to Y_k.Copy X_k to X_k-1. Filter1. LRcore. Filter2. Done. 
Iteration  1. Cpy X_k to Y_k.Copy X_k to X_k-1. Copy G_k-1 to G_k-2. Filter1. LRcore. Filter2. Done. 
Iteration  2. Lambda = 1.00. Copy X_k to X_k-1. Copy G_k-1 to G_k-2. Filter1. LRcore. Filter2. Done. 
Iteration  3. Lambda = 0.00. Copy X_k to X_k-1. Copy G_k-1 to G_k-2. Filter1. LRcore. Filter2. Do

In [10]:
viewer = napari.Viewer()
viewer.add_image(full_deconv, name='deconv', colormap='gray')
napari.run()

In [11]:
angle=32.8
dzdata=0.5*4
dxdata=0.104

print(image.shape)
processing = cupy.asarray(image)
#Swap direction of z axis Needed for 3i data
processing = cupy.flip(processing, 0)
print(processing.shape)

# Step 1
swapped = cupy.swapaxes(processing, 0, 2)
print(swapped.shape)

rotated = cupy.flip(swapped, 2)

# Padded not needed
# Pad to avoid cropping during transform
#angle_rad = angle * np.pi / 180.0
#pad_z = int(rotated.shape[0] * (1 - np.sin(angle_rad)))
#pad_width = [(pad_z, pad_z), (0, 0), (0, 0)]
#padded = np.pad(rotated, pad_width, mode='constant', constant_values=0)
#print('padded',padded.shape)
#padded=rotated
#print('notpadded:',padded.shape)
# Step 2
transformed = scale_and_shear_cupy(rotated, angle, dzdata, dxdata)
print(transformed.shape)

# move to CPU memory
deskew = cupy.asnumpy(transformed)
print(deskew.shape)

(101, 64, 64)
(101, 64, 64)
(64, 64, 101)
(36, 64, 1978)
(36, 64, 1978)


In [12]:
def crop_to_nonzero(data):
    # Find indices of non-zero elements
    nonzero = np.nonzero(data)

    # Get bounding box
    zmin, ymin, xmin = np.min(nonzero, axis=1)
    zmax, ymax, xmax = np.max(nonzero, axis=1) + 1  # +1 to include endpoint

    # Crop the array
    cropped = data[zmin:zmax, ymin:ymax, xmin:xmax]
    return cropped

In [13]:
cropped = crop_to_nonzero(deskew)
print(cropped.shape)  # Should be (4, 6, 8)

(35, 64, 1976)


In [14]:
viewer = napari.Viewer()
#viewer.add_image(image, name='image', colormap='gray')
viewer.add_image(cupy.asnumpy(cropped), name='cropped padded', colormap='gray')
napari.run()

Traceback (most recent call last):
  File "/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/vispy/app/backends/_qt.py", line 928, in paintGL
    self._vispy_canvas.events.draw(region=None)
  File "/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/vispy/scene/canvas.py", line 219, in on_draw
    self._draw_scene()
  File "/home/u1675160/miniconda3/envs/livelattice/lib/python3.10/site-packages/vispy/scene/canvas.py", line 278, in _draw_scene
    self.draw_visual(